# CISC 4900 Project

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import plotly.graph_objects as go
import plotly.io as pio

In [3]:
graph_template = go.layout.Template()
graph_template.layout.plot_bgcolor = 'white'
graph_template.layout.xaxis = dict(
    gridcolor='lightgray'
)
graph_template.layout.yaxis = dict(
    gridcolor='lightgray'
)
pio.templates["graph_template"] = graph_template
pio.templates.default = "graph_template"

## Graduation

In [4]:
graduation_data_2005 = pd.read_excel("https://infohub.nyced.org/docs/default-source/default-document-library/2001-2011-graduation_rates_public_citywide.xlsx", sheet_name=None)
graduation_data_2005.keys()

dict_keys(['NOTES', 'All', 'ELL', 'SWD', 'Ethnicity', 'Gender', 'Gender by Ethnicity', 'Poverty', 'Ever ELL'])

In [5]:
graduation_data_2005['All']

,Category,Cohort Year,Cohort,# Total Cohort,# Grads,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,# Advanced Regents,...,% Local of Cohort,% Local of Grads,# Still Enrolled,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort
0,All Students,2011,4 year August,74363,52409,70.477257,49364,66.382477,94.189926,13639,...,4.094779,5.810071,13927,18.728399,6664,8.961446,741.0,0.996463,484.0,0.650861
1,All Students,2010,4 year August,75524,51681,68.429901,48563,64.301414,93.966835,13421,...,4.128489,6.033165,15127,20.029394,7309,9.677718,781.0,1.034108,484.0,0.640856
2,All Students,2009,4 year August,78721,51970,66.017960,49258,62.572884,94.781609,13535,...,3.445078,5.218395,16577,21.057913,8363,10.623594,948.0,1.204253,693.0,0.880324
3,All Students,2008,4 year August,79719,51573,64.693489,49004,61.470917,95.018715,13264,...,3.222569,4.981288,17063,21.403931,9096,11.410078,1093.0,1.371066,781.0,0.979691
4,All Students,2007,4 year August,79476,52069,65.515373,44216,55.634407,84.918091,13077,...,9.880970,15.081910,15403,19.380693,9631,12.118124,1000.0,1.258242,1272.0,1.600483
5,All Students,2006,4 year August,78346,51006,65.103516,40151,51.248310,78.718193,13001,...,13.855206,21.281811,14975,19.113930,9440,12.049115,1222.0,1.559748,1664.0,2.123912
6,All Students,2005,4 year August,77378,48499,62.678020,35718,46.160408,73.646881,12617,...,16.516323,26.351059,16464,21.277365,9155,11.831529,1336.0,1.726589,1924.0,2.486495
7,All Students,2004,4 year August,75009,45490,60.700001,31976,42.599998,70.300003,12724,...,18.000000,29.700001,17669,23.600000,10126,13.500000,NaN,NaN,NaN,NaN
8,All Students,2011,4 year June,74363,49999,67.236397,47388,63.725239,94.777893,13564,...,3.511155,5.222105,16319,21.945053,6666,8.964135,754.0,1.013945,484.0,0.650861
9,All Students,2010,4 year June,75524,48449,64.150467,45774,60.608547,94.478729,13287,...,3.541920,5.521270,18348,24.294264,7311,9.680367,787.0,1.042053,484.0,0.640856


In [6]:
graduation_data = pd.read_excel("https://infohub.nyced.org/docs/default-source/default-document-library/2024-graduation-rates-public-citywide.xlsx", sheet_name=None)
graduation_data.keys()

dict_keys(['NOTES', 'All', 'ELL', 'Ever ELL', 'SWD', 'Ethnicity', 'Gender', 'Gender by Ethnicity', 'Poverty'])

In [7]:
general_graduation = pd.concat([graduation_data['All'], graduation_data_2005['All']], ignore_index=True)
general_graduation

,Category,Cohort Year,Cohort,# Total Cohort,# Grads,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,# Advanced Regents,...,% Local of Cohort,% Local of Grads,# Still Enrolled,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort
0,All Students,2020,4 year August,68882,57353,83.262680,55757,80.945679,97.217232,15868,...,2.317006,2.782766,7574,10.995616,3237,4.699341,174.0,0.252606,510.0,0.740397
1,All Students,2019,4 year August,69893,58503,83.703659,58295,83.406067,99.644463,18666,...,0.297598,0.355537,6773,9.690527,3748,5.362483,260.0,0.371997,569.0,0.814102
2,All Students,2018,4 year August,70912,59374,83.729126,59110,83.356834,99.555359,24715,...,0.372292,0.444639,6808,9.600632,3831,5.402471,322.0,0.454084,547.0,0.771379
3,All Students,2017,4 year August,72663,60055,82.648666,59683,82.136711,99.380569,26735,...,0.511952,0.619432,8225,11.319379,3545,4.878686,227.0,0.312401,566.0,0.778938
4,All Students,2016,4 year August,73565,58704,79.798821,57734,78.480255,98.347641,22657,...,1.318562,1.652358,9385,12.757425,4338,5.896826,269.0,0.365663,852.0,1.158159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,All Students,2005,6 year June,77994,55277,70.873398,38312,49.121727,69.309113,12783,...,21.751673,30.690884,3425,4.391364,15090,19.347643,2252.0,2.887402,1859.0,2.383517
85,All Students,2004,6 year June,75661,52379,69.228531,34231,45.242596,65.352524,12850,...,23.985937,34.647472,4468,5.905288,15306,20.229710,2156.0,2.849553,1253.0,1.656071
86,All Students,2003,6 year June,74027,48586,65.632812,30483,41.178219,62.740295,11607,...,24.454592,37.259705,4120,5.565537,17517,23.662989,2256.0,3.047537,1450.0,1.958745
87,All Students,2002,6 year June,74511,46067,61.825771,27937,37.493793,60.644279,9768,...,24.691656,39.937481,4729,6.346714,20140,27.029566,2042.0,2.740535,1152.0,1.546080


In [174]:
general_graduation[general_graduation['Cohort Year'] == 2015]

,Category,Cohort Year,Cohort,# Total Cohort,# Grads,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,# Advanced Regents,...,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month,Duration
5,All Students,2015,4 year August,73772,57035,77.312531,51848,70.281410,90.905586,15604,...,13.526813,5745,7.787508,317.0,0.429702,692.0,0.938025,2019,August,4
14,All Students,2015,4 year June,73772,54493,73.866776,49950,67.708618,91.663147,15518,...,16.908854,5790,7.848506,319.0,0.432413,692.0,0.938025,2019,June,4
22,All Students,2015,5 year August,73703,60675,82.323647,54922,74.517998,90.518333,15659,...,5.347136,7215,9.789289,894.0,1.212976,969.0,1.314736,2020,August,5
30,All Students,2015,5 year June,73703,60305,81.821632,54579,74.052620,90.504936,15654,...,5.674125,7344,9.964316,894.0,1.212976,969.0,1.314736,2020,June,5
38,All Students,2015,6 year June,73637,61676,83.756805,55853,75.849098,90.558723,15667,...,3.267379,7578,10.291022,974.0,1.322705,988.0,1.341717,2021,June,6


In [8]:
general_graduation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Category                              89 non-null     object 
 1   Cohort Year                           89 non-null     int64  
 2   Cohort                                89 non-null     object 
 3   # Total Cohort                        89 non-null     int64  
 4   # Grads                               89 non-null     int64  
 5   % Grads                               89 non-null     float64
 6   # Total Regents                       89 non-null     int64  
 7   % Total Regents of Cohort             89 non-null     float64
 8   % Total Regents of Grads              89 non-null     float64
 9   # Advanced Regents                    89 non-null     int64  
 10  % Advanced Regents of Cohort          89 non-null     float64
 11  % Advanced Regents of

In [9]:
general_graduation['Graduation Year'] = general_graduation['Cohort Year'] + general_graduation['Cohort'].str[0].astype(int) #Graduation Year Column

In [10]:
general_graduation['Graduation Month'] = general_graduation['Cohort'].str[7:] #Graduation Month Column
general_graduation['Duration'] = general_graduation['Cohort'].str[0].astype(int)

In [11]:
general_graduation

,Category,Cohort Year,Cohort,# Total Cohort,# Grads,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,# Advanced Regents,...,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month,Duration
0,All Students,2020,4 year August,68882,57353,83.262680,55757,80.945679,97.217232,15868,...,10.995616,3237,4.699341,174.0,0.252606,510.0,0.740397,2024,August,4
1,All Students,2019,4 year August,69893,58503,83.703659,58295,83.406067,99.644463,18666,...,9.690527,3748,5.362483,260.0,0.371997,569.0,0.814102,2023,August,4
2,All Students,2018,4 year August,70912,59374,83.729126,59110,83.356834,99.555359,24715,...,9.600632,3831,5.402471,322.0,0.454084,547.0,0.771379,2022,August,4
3,All Students,2017,4 year August,72663,60055,82.648666,59683,82.136711,99.380569,26735,...,11.319379,3545,4.878686,227.0,0.312401,566.0,0.778938,2021,August,4
4,All Students,2016,4 year August,73565,58704,79.798821,57734,78.480255,98.347641,22657,...,12.757425,4338,5.896826,269.0,0.365663,852.0,1.158159,2020,August,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,All Students,2005,6 year June,77994,55277,70.873398,38312,49.121727,69.309113,12783,...,4.391364,15090,19.347643,2252.0,2.887402,1859.0,2.383517,2011,June,6
85,All Students,2004,6 year June,75661,52379,69.228531,34231,45.242596,65.352524,12850,...,5.905288,15306,20.229710,2156.0,2.849553,1253.0,1.656071,2010,June,6
86,All Students,2003,6 year June,74027,48586,65.632812,30483,41.178219,62.740295,11607,...,5.565537,17517,23.662989,2256.0,3.047537,1450.0,1.958745,2009,June,6
87,All Students,2002,6 year June,74511,46067,61.825771,27937,37.493793,60.644279,9768,...,6.346714,20140,27.029566,2042.0,2.740535,1152.0,1.546080,2008,June,6


In [12]:
general_graduation[general_graduation['Cohort Year']>2019]

,Category,Cohort Year,Cohort,# Total Cohort,# Grads,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,# Advanced Regents,...,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month,Duration
0,All Students,2020,4 year August,68882,57353,83.26268,55757,80.945679,97.217232,15868,...,10.995616,3237,4.699341,174.0,0.252606,510.0,0.740397,2024,August,4
9,All Students,2020,4 year June,68882,55435,80.47821,54159,78.625763,97.698204,15797,...,13.701693,3290,4.776284,175.0,0.254058,510.0,0.740397,2024,June,4


In [13]:
general_graduation = general_graduation[~general_graduation['Cohort Year'].isin([2019,2020])] #Limit dataframe to only students who entered freshman year before 2019
general_graduation

,Category,Cohort Year,Cohort,# Total Cohort,# Grads,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,# Advanced Regents,...,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month,Duration
2,All Students,2018,4 year August,70912,59374,83.729126,59110,83.356834,99.555359,24715,...,9.600632,3831,5.402471,322.0,0.454084,547.0,0.771379,2022,August,4
3,All Students,2017,4 year August,72663,60055,82.648666,59683,82.136711,99.380569,26735,...,11.319379,3545,4.878686,227.0,0.312401,566.0,0.778938,2021,August,4
4,All Students,2016,4 year August,73565,58704,79.798821,57734,78.480255,98.347641,22657,...,12.757425,4338,5.896826,269.0,0.365663,852.0,1.158159,2020,August,4
5,All Students,2015,4 year August,73772,57035,77.312531,51848,70.281410,90.905586,15604,...,13.526813,5745,7.787508,317.0,0.429702,692.0,0.938025,2019,August,4
6,All Students,2014,4 year August,74948,56923,75.949989,52159,69.593582,91.630798,15658,...,15.207877,5614,7.490527,351.0,0.468325,653.0,0.871271,2018,August,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,All Students,2005,6 year June,77994,55277,70.873398,38312,49.121727,69.309113,12783,...,4.391364,15090,19.347643,2252.0,2.887402,1859.0,2.383517,2011,June,6
85,All Students,2004,6 year June,75661,52379,69.228531,34231,45.242596,65.352524,12850,...,5.905288,15306,20.229710,2156.0,2.849553,1253.0,1.656071,2010,June,6
86,All Students,2003,6 year June,74027,48586,65.632812,30483,41.178219,62.740295,11607,...,5.565537,17517,23.662989,2256.0,3.047537,1450.0,1.958745,2009,June,6
87,All Students,2002,6 year June,74511,46067,61.825771,27937,37.493793,60.644279,9768,...,6.346714,20140,27.029566,2042.0,2.740535,1152.0,1.546080,2008,June,6


### Students Who Entered Freshman Year per Year

In [14]:
all_graduates = general_graduation.groupby(['Cohort Year', 'Duration'])['# Grads'].max().reset_index()
all_graduates

,Cohort Year,Duration,# Grads
0,2001,4,34503
1,2001,5,41317
2,2001,6,43351
3,2002,4,36552
4,2002,5,43835
5,2002,6,46067
6,2003,4,38990
7,2003,5,46265
8,2003,6,48586
9,2004,4,45490


In [15]:
all_graduates = general_graduation.groupby(['Cohort Year', 'Duration'])['# Grads'].max().reset_index()
all_graduates = all_graduates.merge(general_graduation.drop(columns=['Cohort Year', 'Duration']), on='# Grads')
all_graduates

,Cohort Year,Duration,# Grads,Category,Cohort,# Total Cohort,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,...,# Still Enrolled,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month
0,2001,4,34503,All Students,4 year June,74143,46.535748,22197,29.938093,64.333534,...,21094,28.450426,16337,22.034447,1895.0,2.555872,294.0,0.396531,2005,June
1,2001,5,41317,All Students,5 year June,74143,55.726097,23925,32.268723,57.905945,...,8335,11.241790,21382,28.838865,2739.0,3.694212,294.0,0.396531,2006,June
2,2001,6,43351,All Students,6 year June,74143,58.469444,24338,32.825756,56.141727,...,3896,5.254711,23538,31.746759,2988.0,4.030050,294.0,0.396531,2007,June
3,2002,4,36552,All Students,4 year June,74511,49.055843,25098,33.683617,68.663818,...,21721,29.151400,13758,18.464388,1010.0,1.355505,1102.0,1.478976,2006,June
4,2002,5,43835,All Students,5 year June,74511,58.830238,27478,36.877777,62.685070,...,8927,11.980782,18434,24.739971,1814.0,2.434540,1120.0,1.503134,2007,June
5,2002,6,46067,All Students,6 year June,74511,61.825771,27937,37.493793,60.644279,...,4729,6.346714,20140,27.029566,2042.0,2.740535,1152.0,1.546080,2008,June
6,2003,4,38990,All Students,4 year June,73888,52.769054,27445,37.144054,70.389847,...,21353,28.899145,11704,15.840191,1113.0,1.506334,633.0,0.856702,2007,June
7,2003,5,46265,All Students,5 year June,73888,62.615040,29887,40.449059,64.599586,...,8520,11.530966,16095,21.782969,2129.0,2.881388,781.0,1.057005,2008,June
8,2003,6,48586,All Students,6 year June,74027,65.632812,30483,41.178219,62.740295,...,4120,5.565537,17517,23.662989,2256.0,3.047537,1450.0,1.958745,2009,June
9,2004,4,45490,All Students,4 year August,75009,60.700001,31976,42.599998,70.300003,...,17669,23.600000,10126,13.500000,NaN,NaN,NaN,NaN,2008,August


In [16]:
def get_graduation_total(df):
    graduation_total_df = df.groupby('Cohort Year')[['# Grads','# Total Cohort']].sum().reset_index()
    graduation_total_df['Average Grad %'] = graduation_total_df['# Grads'] / graduation_total_df['# Total Cohort'] * 100
    return graduation_total_df

In [18]:
cohort_total_students_per_year = get_graduation_total(all_graduates)
cohort_total_students_per_year

,Cohort Year,# Grads,# Total Cohort,Average Grad %
0,2001,119171,222429,53.577097
1,2002,126454,223533,56.570618
2,2003,133841,221803,60.342286
3,2004,147828,226295,65.325350
4,2005,156764,233519,67.131154
5,2006,163835,235548,69.554825
6,2007,165873,237396,69.871860
7,2008,165155,238314,69.301426
8,2009,167256,235323,71.075076
9,2010,165446,226020,73.199717


In [127]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cohort_total_students_per_year['Cohort Year'], y=cohort_total_students_per_year['# Total Cohort']))

fig.update_layout(
    title="Students Who Entered 9th Grade Every Year",
    xaxis_title="Cohort Year",
    yaxis_title="# Students",
    showlegend=False,
    xaxis=dict(
        tickvals=cohort_total_students_per_year['Cohort Year'], 
        ticktext=cohort_total_students_per_year['Cohort Year'],
    )
)

In [128]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=cohort_total_students_per_year['Cohort Year'], y=cohort_total_students_per_year['Average Grad %']))

fig.update_layout(
    title="% Graduates per Cohort",
    xaxis_title="Cohort Year",
    yaxis_title="% Graduates",
    showlegend=False,
    xaxis=dict(
        tickvals=cohort_total_students_per_year['Cohort Year'], 
        ticktext=cohort_total_students_per_year['Cohort Year'],
    )
)

### 4-Year Graduates

In [22]:
four_year_graduates = all_graduates[all_graduates['Duration'] == 4].reset_index(drop=True)
four_year_graduates

,Cohort Year,Duration,# Grads,Category,Cohort,# Total Cohort,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,...,# Still Enrolled,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month
0,2001,4,34503,All Students,4 year June,74143,46.535748,22197,29.938093,64.333534,...,21094,28.450426,16337,22.034447,1895.0,2.555872,294.0,0.396531,2005,June
1,2002,4,36552,All Students,4 year June,74511,49.055843,25098,33.683617,68.663818,...,21721,29.151400,13758,18.464388,1010.0,1.355505,1102.0,1.478976,2006,June
2,2003,4,38990,All Students,4 year June,73888,52.769054,27445,37.144054,70.389847,...,21353,28.899145,11704,15.840191,1113.0,1.506334,633.0,0.856702,2007,June
3,2004,4,45490,All Students,4 year August,75009,60.700001,31976,42.599998,70.300003,...,17669,23.600000,10126,13.500000,NaN,NaN,NaN,NaN,2008,August
4,2005,4,48499,All Students,4 year August,77378,62.678020,35718,46.160408,73.646881,...,16464,21.277365,9155,11.831529,1336.0,1.726589,1924.0,2.486495,2009,August
5,2006,4,51006,All Students,4 year August,78346,65.103516,40151,51.248310,78.718193,...,14975,19.113930,9440,12.049115,1222.0,1.559748,1664.0,2.123912,2010,August
6,2007,4,52069,All Students,4 year August,79476,65.515373,44216,55.634407,84.918091,...,15403,19.380693,9631,12.118124,1000.0,1.258242,1272.0,1.600483,2011,August
7,2008,4,51573,All Students,4 year August,79719,64.693489,49004,61.470917,95.018715,...,17063,21.403931,9096,11.410078,1093.0,1.371066,781.0,0.979691,2012,August
8,2009,4,51970,All Students,4 year August,78721,66.017960,49258,62.572884,94.781609,...,16577,21.057913,8363,10.623594,948.0,1.204253,693.0,0.880324,2013,August
9,2010,4,51681,All Students,4 year August,75524,68.429901,48563,64.301414,93.966835,...,15127,20.029394,7309,9.677718,781.0,1.034108,484.0,0.640856,2014,August


In [23]:
four_year_graduates_june = general_graduation[(general_graduation['Duration'] == 4) & (general_graduation['Graduation Month'] == 'June')].sort_values(by='Cohort Year').reset_index(drop=True)
four_year_graduates_june

,Category,Cohort Year,Cohort,# Total Cohort,# Grads,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,# Advanced Regents,...,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month,Duration
0,All Students,2001,4 year June,74143,34503,46.535748,22197,29.938093,64.333534,9249,...,28.450426,16337,22.034447,1895.0,2.555872,294.0,0.396531,2005,June,4
1,All Students,2002,4 year June,74511,36552,49.055843,25098,33.683617,68.663818,9435,...,29.151400,13758,18.464388,1010.0,1.355505,1102.0,1.478976,2006,June,4
2,All Students,2003,4 year June,73888,38990,52.769054,27445,37.144054,70.389847,11356,...,28.899145,11704,15.840191,1113.0,1.506334,633.0,0.856702,2007,June,4
3,All Students,2004,4 year June,75009,42313,56.410564,30652,40.864429,72.441093,12592,...,27.788666,10127,13.501046,1409.0,1.878441,309.0,0.411951,2008,June,4
4,All Students,2005,4 year June,77378,45615,58.950863,34489,44.572102,75.608902,12512,...,24.998062,9157,11.834113,1339.0,1.730466,1924.0,2.486495,2009,June,4
5,All Students,2006,4 year June,78346,47786,60.993542,38570,49.230339,80.714020,12885,...,23.149874,9497,12.121870,1222.0,1.559748,1665.0,2.125188,2010,June,4
6,All Students,2007,4 year June,79476,48380,60.873722,42106,52.979515,87.031830,12958,...,23.856258,9740,12.255272,1060.0,1.333736,1233.0,1.551412,2011,June,4
7,All Students,2008,4 year June,79719,48173,60.428505,46024,57.732788,95.538994,13175,...,25.571068,9131,11.453982,1136.0,1.425005,781.0,0.979691,2012,June,4
8,All Students,2009,4 year June,78721,48236,61.274628,45989,58.420242,95.341652,13400,...,25.718678,8370,10.632487,1004.0,1.275390,693.0,0.880324,2013,June,4
9,All Students,2010,4 year June,75524,48449,64.150467,45774,60.608547,94.478729,13287,...,24.294264,7311,9.680367,787.0,1.042053,484.0,0.640856,2014,June,4


In [135]:
def plot_grad(df):
    fig = go.Figure()

    fig.add_trace(go.Bar(x=df['Cohort Year'], y=df['June Grad %'], name='June'))
    fig.add_trace(go.Bar(x=df['Cohort Year'], y=df['August Grad %'], name='August'))

    fig.update_layout(
        xaxis_title="Cohort Year",
        yaxis_title="% Graduates",
        xaxis=dict(
            tickvals=df['Cohort Year'], 
            ticktext=df['Cohort Year']
        ),
        barmode='stack'
    )

    return fig

In [25]:
four_year_graduates

,Cohort Year,Duration,# Grads,Category,Cohort,# Total Cohort,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,...,# Still Enrolled,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month
0,2001,4,34503,All Students,4 year June,74143,46.535748,22197,29.938093,64.333534,...,21094,28.450426,16337,22.034447,1895.0,2.555872,294.0,0.396531,2005,June
1,2002,4,36552,All Students,4 year June,74511,49.055843,25098,33.683617,68.663818,...,21721,29.151400,13758,18.464388,1010.0,1.355505,1102.0,1.478976,2006,June
2,2003,4,38990,All Students,4 year June,73888,52.769054,27445,37.144054,70.389847,...,21353,28.899145,11704,15.840191,1113.0,1.506334,633.0,0.856702,2007,June
3,2004,4,45490,All Students,4 year August,75009,60.700001,31976,42.599998,70.300003,...,17669,23.600000,10126,13.500000,NaN,NaN,NaN,NaN,2008,August
4,2005,4,48499,All Students,4 year August,77378,62.678020,35718,46.160408,73.646881,...,16464,21.277365,9155,11.831529,1336.0,1.726589,1924.0,2.486495,2009,August
5,2006,4,51006,All Students,4 year August,78346,65.103516,40151,51.248310,78.718193,...,14975,19.113930,9440,12.049115,1222.0,1.559748,1664.0,2.123912,2010,August
6,2007,4,52069,All Students,4 year August,79476,65.515373,44216,55.634407,84.918091,...,15403,19.380693,9631,12.118124,1000.0,1.258242,1272.0,1.600483,2011,August
7,2008,4,51573,All Students,4 year August,79719,64.693489,49004,61.470917,95.018715,...,17063,21.403931,9096,11.410078,1093.0,1.371066,781.0,0.979691,2012,August
8,2009,4,51970,All Students,4 year August,78721,66.017960,49258,62.572884,94.781609,...,16577,21.057913,8363,10.623594,948.0,1.204253,693.0,0.880324,2013,August
9,2010,4,51681,All Students,4 year August,75524,68.429901,48563,64.301414,93.966835,...,15127,20.029394,7309,9.677718,781.0,1.034108,484.0,0.640856,2014,August


In [26]:
four_year_graduates_june.sort_values(by='Cohort Year').reset_index(drop=True)

,Category,Cohort Year,Cohort,# Total Cohort,# Grads,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,# Advanced Regents,...,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month,Duration
0,All Students,2001,4 year June,74143,34503,46.535748,22197,29.938093,64.333534,9249,...,28.450426,16337,22.034447,1895.0,2.555872,294.0,0.396531,2005,June,4
1,All Students,2002,4 year June,74511,36552,49.055843,25098,33.683617,68.663818,9435,...,29.151400,13758,18.464388,1010.0,1.355505,1102.0,1.478976,2006,June,4
2,All Students,2003,4 year June,73888,38990,52.769054,27445,37.144054,70.389847,11356,...,28.899145,11704,15.840191,1113.0,1.506334,633.0,0.856702,2007,June,4
3,All Students,2004,4 year June,75009,42313,56.410564,30652,40.864429,72.441093,12592,...,27.788666,10127,13.501046,1409.0,1.878441,309.0,0.411951,2008,June,4
4,All Students,2005,4 year June,77378,45615,58.950863,34489,44.572102,75.608902,12512,...,24.998062,9157,11.834113,1339.0,1.730466,1924.0,2.486495,2009,June,4
5,All Students,2006,4 year June,78346,47786,60.993542,38570,49.230339,80.714020,12885,...,23.149874,9497,12.121870,1222.0,1.559748,1665.0,2.125188,2010,June,4
6,All Students,2007,4 year June,79476,48380,60.873722,42106,52.979515,87.031830,12958,...,23.856258,9740,12.255272,1060.0,1.333736,1233.0,1.551412,2011,June,4
7,All Students,2008,4 year June,79719,48173,60.428505,46024,57.732788,95.538994,13175,...,25.571068,9131,11.453982,1136.0,1.425005,781.0,0.979691,2012,June,4
8,All Students,2009,4 year June,78721,48236,61.274628,45989,58.420242,95.341652,13400,...,25.718678,8370,10.632487,1004.0,1.275390,693.0,0.880324,2013,June,4
9,All Students,2010,4 year June,75524,48449,64.150467,45774,60.608547,94.478729,13287,...,24.294264,7311,9.680367,787.0,1.042053,484.0,0.640856,2014,June,4


In [27]:
four_year_grad_perc = four_year_graduates[['Cohort Year', '# Grads', '# Total Cohort']].reset_index(drop=True)
four_year_grad_perc['June Grads'] = four_year_graduates_june['# Grads']
four_year_grad_perc['August Grads'] = four_year_grad_perc['# Grads'] - four_year_grad_perc['June Grads']
four_year_grad_perc['June Grad %'] = four_year_grad_perc['June Grads'] / four_year_grad_perc['# Total Cohort'] * 100
four_year_grad_perc['August Grad %'] = four_year_grad_perc['August Grads'] / four_year_grad_perc['# Total Cohort'] * 100
four_year_grad_perc

,Cohort Year,# Grads,# Total Cohort,June Grads,August Grads,June Grad %,August Grad %
0,2001,34503,74143,34503,0,46.535748,0.000000
1,2002,36552,74511,36552,0,49.055844,0.000000
2,2003,38990,73888,38990,0,52.769056,0.000000
3,2004,45490,75009,42313,3177,56.410564,4.235492
4,2005,48499,77378,45615,2884,58.950865,3.727158
5,2006,51006,78346,47786,3220,60.993541,4.109974
6,2007,52069,79476,48380,3689,60.873723,4.641653
7,2008,51573,79719,48173,3400,60.428505,4.264981
8,2009,51970,78721,48236,3734,61.274628,4.743334
9,2010,51681,75524,48449,3232,64.150469,4.279434


In [136]:
fig = plot_grad(four_year_grad_perc)
fig.update_layout(
    title="% of 4-Year High School Graduates"
)

### 5-Year Graduates

In [29]:
five_year_graduates = all_graduates[all_graduates['Duration'] == 5].reset_index(drop=True)
five_year_graduates

,Cohort Year,Duration,# Grads,Category,Cohort,# Total Cohort,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,...,# Still Enrolled,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month
0,2001,5,41317,All Students,5 year June,74143,55.726097,23925,32.268723,57.905945,...,8335,11.241790,21382,28.838865,2739.0,3.694212,294.0,0.396531,2006,June
1,2002,5,43835,All Students,5 year June,74511,58.830238,27478,36.877777,62.685070,...,8927,11.980782,18434,24.739971,1814.0,2.434540,1120.0,1.503134,2007,June
2,2003,5,46265,All Students,5 year June,73888,62.615040,29887,40.449059,64.599586,...,8520,11.530966,16095,21.782969,2129.0,2.881388,781.0,1.057005,2008,June
3,2004,5,49959,All Students,5 year June,75625,66.061485,33539,44.349091,67.133049,...,7365,9.738843,13786,18.229422,2104.0,2.782149,2401.0,3.174876,2009,June
4,2005,5,52988,All Students,5 year June,78147,67.805550,37611,48.128529,70.980225,...,6666,8.530078,14259,18.246382,2122.0,2.715395,2040.0,2.610465,2010,June
5,2006,5,55708,All Students,5 year August,79156,70.377480,42189,53.298550,75.732391,...,5650,7.137804,13877,17.531204,2228.0,2.814695,1611.0,2.035222,2011,August
6,2007,5,56153,All Students,5 year August,78922,71.149994,46104,58.417171,82.104248,...,4962,6.287220,14467,18.330757,1949.0,2.469527,1282.0,1.624389,2012,August
7,2008,5,56007,All Students,5 year August,79362,70.571556,52678,66.376854,94.056099,...,7700,9.702376,12572,15.841334,2008.0,2.530178,881.0,1.110103,2013,August
8,2009,5,56781,All Students,5 year August,78315,72.503349,53204,67.935898,93.700356,...,7540,9.627786,11731,14.979251,1228.0,1.568027,815.0,1.040669,2014,August
9,2010,5,56132,All Students,5 year August,75242,74.601952,52054,69.182106,92.734985,...,6602,8.774355,10495,13.948327,1216.0,1.616119,614.0,0.816034,2015,August


In [30]:
five_year_graduates_june = general_graduation[(general_graduation['Duration'] == 5) & (general_graduation['Graduation Month'] == 'June')].sort_values(by='Cohort Year').reset_index(drop=True)
five_year_graduates_june

,Category,Cohort Year,Cohort,# Total Cohort,# Grads,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,# Advanced Regents,...,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month,Duration
0,All Students,2001,5 year June,74143,41317,55.726097,23925,32.268723,57.905945,9449,...,11.241790,21382,28.838865,2739.0,3.694212,294.0,0.396531,2006,June,5
1,All Students,2002,5 year June,74511,43835,58.830238,27478,36.877777,62.685070,9745,...,11.980782,18434,24.739971,1814.0,2.434540,1120.0,1.503134,2007,June,5
2,All Students,2003,5 year June,73888,46265,62.615040,29887,40.449059,64.599586,11594,...,11.530966,16095,21.782969,2129.0,2.881388,781.0,1.057005,2008,June,5
3,All Students,2004,5 year June,75625,49959,66.061485,33539,44.349091,67.133049,12841,...,9.738843,13786,18.229422,2104.0,2.782149,2401.0,3.174876,2009,June,5
4,All Students,2005,5 year June,78147,52988,67.805550,37611,48.128529,70.980225,12774,...,8.530078,14259,18.246382,2122.0,2.715395,2040.0,2.610465,2010,June,5
5,All Students,2006,5 year June,79156,54936,69.402191,41918,52.956188,76.303337,13075,...,7.868008,14103,17.816717,2228.0,2.814695,1579.0,1.994795,2011,June,5
6,All Students,2007,5 year June,78922,55404,70.200958,45770,57.993969,82.611366,13023,...,6.977776,14658,18.572767,1962.0,2.485999,1282.0,1.624389,2012,June,5
7,All Students,2008,5 year June,79362,55132,69.469017,51954,65.464577,94.235649,13313,...,10.694035,12639,15.925758,2024.0,2.550339,881.0,1.110103,2013,June,5
8,All Students,2009,5 year June,78315,55882,71.355423,52471,66.999939,93.896065,13584,...,10.716976,11766,15.023942,1238.0,1.580796,815.0,1.040669,2014,June,5
9,All Students,2010,5 year June,75242,55494,73.754021,51552,68.514923,92.896530,13467,...,9.571782,10528,13.992186,1221.0,1.622764,614.0,0.816034,2015,June,5


In [31]:
five_year_grad_perc = five_year_graduates[['Cohort Year', '# Grads', '# Total Cohort']].reset_index(drop=True)
five_year_grad_perc['June Grads'] = five_year_graduates_june['# Grads']
five_year_grad_perc['August Grads'] = five_year_grad_perc['# Grads'] - five_year_grad_perc['June Grads']
five_year_grad_perc['June Grad %'] = five_year_grad_perc['June Grads'] / five_year_grad_perc['# Total Cohort'] * 100
five_year_grad_perc['August Grad %'] = five_year_grad_perc['August Grads'] / five_year_grad_perc['# Total Cohort'] * 100
five_year_grad_perc

,Cohort Year,# Grads,# Total Cohort,June Grads,August Grads,June Grad %,August Grad %
0,2001,41317,74143,41317,0,55.726097,0.000000
1,2002,43835,74511,43835,0,58.830240,0.000000
2,2003,46265,73888,46265,0,62.615039,0.000000
3,2004,49959,75625,49959,0,66.061488,0.000000
4,2005,52988,78147,52988,0,67.805546,0.000000
5,2006,55708,79156,54936,772,69.402193,0.975289
6,2007,56153,78922,55404,749,70.200958,0.949038
7,2008,56007,79362,55132,875,69.469015,1.102543
8,2009,56781,78315,55882,899,71.355424,1.147928
9,2010,56132,75242,55494,638,73.754020,0.847931


In [32]:
five_year_grad_perc['Cohort Year'].values

array([2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [138]:
fig = plot_grad(five_year_grad_perc)
fig.update_layout(
    title="% of 5-Year High School Graduates"
)

### 6-Year Graduates

In [34]:
six_year_graduates = all_graduates[all_graduates['Duration'] == 6].reset_index(drop=True).sort_values(by='Cohort Year')
six_year_graduates

,Cohort Year,Duration,# Grads,Category,Cohort,# Total Cohort,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,...,# Still Enrolled,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month
0,2001,6,43351,All Students,6 year June,74143,58.469444,24338,32.825756,56.141727,...,3896,5.254711,23538,31.746759,2988.0,4.030050,294.0,0.396531,2007,June
1,2002,6,46067,All Students,6 year June,74511,61.825771,27937,37.493793,60.644279,...,4729,6.346714,20140,27.029566,2042.0,2.740535,1152.0,1.546080,2008,June
2,2003,6,48586,All Students,6 year June,74027,65.632812,30483,41.178219,62.740295,...,4120,5.565537,17517,23.662989,2256.0,3.047537,1450.0,1.958745,2009,June
3,2004,6,52379,All Students,6 year June,75661,69.228531,34231,45.242596,65.352524,...,4468,5.905288,15306,20.229710,2156.0,2.849553,1253.0,1.656071,2010,June
4,2005,6,55277,All Students,6 year June,77994,70.873398,38312,49.121727,69.309113,...,3425,4.391364,15090,19.347643,2252.0,2.887402,1859.0,2.383517,2011,June
5,2006,6,57121,All Students,6 year June,78046,73.188889,42689,54.697231,74.734337,...,2484,3.182738,14618,18.729980,2174.0,2.785537,1541.0,1.974477,2012,June
6,2007,6,57651,All Students,6 year June,78998,72.977798,46967,59.453403,81.467796,...,2787,3.527937,15040,19.038456,2148.0,2.719056,1228.0,1.554470,2013,June
7,2008,6,57575,All Students,6 year June,79233,72.665428,53861,67.977989,93.549286,...,4351,5.491399,13545,17.095150,2699.0,3.406409,869.0,1.096765,2014,June
8,2009,6,58505,All Students,6 year June,78287,74.731438,54516,69.636086,93.181778,...,4541,5.800452,12818,16.373089,1332.0,1.701432,850.0,1.085749,2015,June
9,2010,6,57633,All Students,6 year June,75254,76.584633,53141,70.615517,92.205856,...,4022,5.344566,11446,15.209823,1309.0,1.739442,617.0,0.819890,2016,June


In [35]:
six_year_graduates_june = general_graduation[(general_graduation['Duration'] == 6) & (general_graduation['Graduation Month'] == 'June')].sort_values(by='Cohort Year').reset_index(drop=True)
six_year_graduates_june

,Category,Cohort Year,Cohort,# Total Cohort,# Grads,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,# Advanced Regents,...,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month,Duration
0,All Students,2001,6 year June,74143,43351,58.469444,24338,32.825756,56.141727,9473,...,5.254711,23538,31.746759,2988.0,4.030050,294.0,0.396531,2007,June,6
1,All Students,2002,6 year June,74511,46067,61.825771,27937,37.493793,60.644279,9768,...,6.346714,20140,27.029566,2042.0,2.740535,1152.0,1.546080,2008,June,6
2,All Students,2003,6 year June,74027,48586,65.632812,30483,41.178219,62.740295,11607,...,5.565537,17517,23.662989,2256.0,3.047537,1450.0,1.958745,2009,June,6
3,All Students,2004,6 year June,75661,52379,69.228531,34231,45.242596,65.352524,12850,...,5.905288,15306,20.229710,2156.0,2.849553,1253.0,1.656071,2010,June,6
4,All Students,2005,6 year June,77994,55277,70.873398,38312,49.121727,69.309113,12783,...,4.391364,15090,19.347643,2252.0,2.887402,1859.0,2.383517,2011,June,6
5,All Students,2006,6 year June,78046,57121,73.188889,42689,54.697231,74.734337,13064,...,3.182738,14618,18.729980,2174.0,2.785537,1541.0,1.974477,2012,June,6
6,All Students,2007,6 year June,78998,57651,72.977798,46967,59.453403,81.467796,13109,...,3.527937,15040,19.038456,2148.0,2.719056,1228.0,1.554470,2013,June,6
7,All Students,2008,6 year June,79233,57575,72.665428,53861,67.977989,93.549286,13324,...,5.491399,13545,17.095150,2699.0,3.406409,869.0,1.096765,2014,June,6
8,All Students,2009,6 year June,78287,58505,74.731438,54516,69.636086,93.181778,13592,...,5.800452,12818,16.373089,1332.0,1.701432,850.0,1.085749,2015,June,6
9,All Students,2010,6 year June,75254,57633,76.584633,53141,70.615517,92.205856,13473,...,5.344566,11446,15.209823,1309.0,1.739442,617.0,0.819890,2016,June,6


In [36]:
six_year_grad_perc = six_year_graduates[['Cohort Year', '# Grads', '# Total Cohort']].reset_index(drop=True)
six_year_grad_perc['June Grads'] = six_year_graduates_june['# Grads']
six_year_grad_perc['August Grads'] = six_year_grad_perc['# Grads'] - six_year_grad_perc['June Grads']
six_year_grad_perc['June Grad %'] = six_year_grad_perc['June Grads'] / six_year_grad_perc['# Total Cohort'] * 100
six_year_grad_perc['August Grad %'] = six_year_grad_perc['August Grads'] / six_year_grad_perc['# Total Cohort'] * 100
six_year_grad_perc

,Cohort Year,# Grads,# Total Cohort,June Grads,August Grads,June Grad %,August Grad %
0,2001,43351,74143,43351,0,58.469444,0.000000
1,2002,46067,74511,46067,0,61.825771,0.000000
2,2003,48586,74027,48586,0,65.632810,0.000000
3,2004,52379,75661,52379,0,69.228533,0.000000
4,2005,55277,77994,55277,0,70.873401,0.000000
5,2006,57121,78046,57121,0,73.188889,0.000000
6,2007,57651,78998,57651,0,72.977797,0.000000
7,2008,57575,79233,57575,0,72.665430,0.000000
8,2009,58505,78287,58505,0,74.731437,0.000000
9,2010,57633,75254,57633,0,76.584633,0.000000


In [139]:
fig = plot_grad(six_year_grad_perc)
fig.update_layout(
    title="% of 6-Year High School Graduates"
)

In [38]:
cohort_total_students_per_year['4-year Grads'] = four_year_graduates['# Grads']
cohort_total_students_per_year['5-year Grads'] = five_year_graduates['# Grads']
cohort_total_students_per_year['6-year Grads'] = six_year_graduates['# Grads']
cohort_total_students_per_year['4-year Grad %'] = cohort_total_students_per_year['4-year Grads'] / cohort_total_students_per_year['# Total Cohort'] * 100
cohort_total_students_per_year['5-year Grad %'] = cohort_total_students_per_year['5-year Grads'] / cohort_total_students_per_year['# Total Cohort'] * 100
cohort_total_students_per_year['6-year Grad %'] = cohort_total_students_per_year['6-year Grads'] / cohort_total_students_per_year['# Total Cohort'] * 100
cohort_total_students_per_year

,Cohort Year,# Grads,# Total Cohort,Average Grad %,4-year Grads,5-year Grads,6-year Grads,4-year Grad %,5-year Grad %,6-year Grad %
0,2001,119171,222429,53.577097,34503,41317,43351,15.511916,18.575366,19.489815
1,2002,126454,223533,56.570618,36552,43835,46067,16.351948,19.610080,20.608590
2,2003,133841,221803,60.342286,38990,46265,48586,17.578662,20.858600,21.905024
3,2004,147828,226295,65.325350,45490,49959,52379,20.102079,22.076935,23.146336
4,2005,156764,233519,67.131154,48499,52988,55277,20.768760,22.691087,23.671307
5,2006,163835,235548,69.554825,51006,55708,57121,21.654185,23.650381,24.250259
6,2007,165873,237396,69.871860,52069,56153,57651,21.933394,23.653726,24.284739
7,2008,165155,238314,69.301426,51573,56007,57575,21.640776,23.501347,24.159302
8,2009,167256,235323,71.075076,51970,56781,58505,22.084539,24.128963,24.861573
9,2010,165446,226020,73.199717,51681,56132,57633,22.865676,24.834970,25.499071


In [123]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=cohort_total_students_per_year['Cohort Year'], y=cohort_total_students_per_year['4-year Grad %'], mode='lines+markers', name='4 Years'))
fig.add_trace(go.Scatter(x=cohort_total_students_per_year['Cohort Year'], y=cohort_total_students_per_year['5-year Grad %'], mode='lines+markers', name='5 Years'))
fig.add_trace(go.Scatter(x=cohort_total_students_per_year['Cohort Year'], y=cohort_total_students_per_year['6-year Grad %'], mode='lines+markers', name='6 Years'))

fig.update_layout(
    title="Graduate Distribution Based on Duration",
    xaxis_title="Cohort Year",
    yaxis_title="% Graduates",
    xaxis=dict(
        tickvals=cohort_total_students_per_year['Cohort Year'], 
        ticktext=cohort_total_students_per_year['Cohort Year']
    )
)

### Predictions for the next 5 years

In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [41]:
cohort_total_students_per_year

,Cohort Year,# Grads,# Total Cohort,Average Grad %,4-year Grads,5-year Grads,6-year Grads,4-year Grad %,5-year Grad %,6-year Grad %
0,2001,119171,222429,53.577097,34503,41317,43351,15.511916,18.575366,19.489815
1,2002,126454,223533,56.570618,36552,43835,46067,16.351948,19.610080,20.608590
2,2003,133841,221803,60.342286,38990,46265,48586,17.578662,20.858600,21.905024
3,2004,147828,226295,65.325350,45490,49959,52379,20.102079,22.076935,23.146336
4,2005,156764,233519,67.131154,48499,52988,55277,20.768760,22.691087,23.671307
5,2006,163835,235548,69.554825,51006,55708,57121,21.654185,23.650381,24.250259
6,2007,165873,237396,69.871860,52069,56153,57651,21.933394,23.653726,24.284739
7,2008,165155,238314,69.301426,51573,56007,57575,21.640776,23.501347,24.159302
8,2009,167256,235323,71.075076,51970,56781,58505,22.084539,24.128963,24.861573
9,2010,165446,226020,73.199717,51681,56132,57633,22.865676,24.834970,25.499071


In [112]:
x = cohort_total_students_per_year['Cohort Year'].values
y = cohort_total_students_per_year['Average Grad %'].values
np.corrcoef(x, y)[0,1]

0.981405075121107

In [113]:
x = x.reshape(-1,1)
model = LinearRegression()
model.fit(x, y)

LinearRegression()

In [114]:
def get_future_years(df):
    future_years = []
    for i in range(1,6):
        future_years.append(df['Cohort Year'].max() + i)

    future_years = np.array(future_years)
    return future_years

In [115]:
future_years = get_future_years(cohort_total_students_per_year)
future_preds = model.predict(future_years.reshape(-1,1))
current_trend = model.predict(x)

In [116]:
future_preds

array([88.57367452, 90.29514296, 92.0166114 , 93.73807984, 95.45954828])

In [117]:
current_trend

array([57.58724261, 59.30871105, 61.03017949, 62.75164793, 64.47311637,
       66.19458481, 67.91605325, 69.63752169, 71.35899013, 73.08045857,
       74.80192701, 76.52339544, 78.24486388, 79.96633232, 81.68780076,
       83.4092692 , 85.13073764, 86.85220608])

In [118]:
r2_score(y, current_trend)

0.9631559214734645

In [140]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=cohort_total_students_per_year['Cohort Year'], y=cohort_total_students_per_year['Average Grad %'], mode='markers', name='Current Data'))
fig.add_trace(go.Scatter(x=cohort_total_students_per_year['Cohort Year'], y=current_trend, mode='lines', name='Current Trend'))
fig.add_trace(go.Scatter(x=future_years, y=future_preds, mode='markers', marker=dict(symbol="x", size=10), name='Predictions'))
fig.update_layout(
    title="Graduation Rate Predictions for the Next 5 Years",
    xaxis_title="Cohort Year",
    yaxis_title="% Graduates",
    xaxis=dict(
        tickvals=np.append(six_year_graduates['Cohort Year'].values,future_years), 
        ticktext=np.append(six_year_graduates['Cohort Year'].values,future_years)
    )
)

#### Four Years

In [51]:
four_year_graduates

,Cohort Year,Duration,# Grads,Category,Cohort,# Total Cohort,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,...,# Still Enrolled,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month
0,2001,4,34503,All Students,4 year June,74143,46.535748,22197,29.938093,64.333534,...,21094,28.450426,16337,22.034447,1895.0,2.555872,294.0,0.396531,2005,June
1,2002,4,36552,All Students,4 year June,74511,49.055843,25098,33.683617,68.663818,...,21721,29.151400,13758,18.464388,1010.0,1.355505,1102.0,1.478976,2006,June
2,2003,4,38990,All Students,4 year June,73888,52.769054,27445,37.144054,70.389847,...,21353,28.899145,11704,15.840191,1113.0,1.506334,633.0,0.856702,2007,June
3,2004,4,45490,All Students,4 year August,75009,60.700001,31976,42.599998,70.300003,...,17669,23.600000,10126,13.500000,NaN,NaN,NaN,NaN,2008,August
4,2005,4,48499,All Students,4 year August,77378,62.678020,35718,46.160408,73.646881,...,16464,21.277365,9155,11.831529,1336.0,1.726589,1924.0,2.486495,2009,August
5,2006,4,51006,All Students,4 year August,78346,65.103516,40151,51.248310,78.718193,...,14975,19.113930,9440,12.049115,1222.0,1.559748,1664.0,2.123912,2010,August
6,2007,4,52069,All Students,4 year August,79476,65.515373,44216,55.634407,84.918091,...,15403,19.380693,9631,12.118124,1000.0,1.258242,1272.0,1.600483,2011,August
7,2008,4,51573,All Students,4 year August,79719,64.693489,49004,61.470917,95.018715,...,17063,21.403931,9096,11.410078,1093.0,1.371066,781.0,0.979691,2012,August
8,2009,4,51970,All Students,4 year August,78721,66.017960,49258,62.572884,94.781609,...,16577,21.057913,8363,10.623594,948.0,1.204253,693.0,0.880324,2013,August
9,2010,4,51681,All Students,4 year August,75524,68.429901,48563,64.301414,93.966835,...,15127,20.029394,7309,9.677718,781.0,1.034108,484.0,0.640856,2014,August


In [146]:
x = four_year_graduates['Cohort Year'].values
y = four_year_graduates['% Grads'].values
np.corrcoef(x,y)[0,1]

0.9749114015280054

In [147]:
x = x.reshape(-1, 1)

In [148]:
model.fit(x, y)

LinearRegression()

In [149]:
future_years = get_future_years(four_year_graduates)
current_trend = model.predict(x)
four_year_grad_preds = model.predict(future_years.reshape(-1, 1))

In [150]:
r2_score(y, current_trend)

0.9504522408293004

In [151]:
four_year_grad_preds

array([86.44196033, 88.41423751, 90.38651469, 92.35879186, 94.33106904])

In [152]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=four_year_graduates['Cohort Year'], y=four_year_graduates['% Grads'], mode='markers', name='Current Data'))
fig.add_trace(go.Scatter(x=four_year_graduates['Cohort Year'], y=current_trend, mode='lines', name='Current Trend'))
fig.add_trace(go.Scatter(x=future_years, y=four_year_grad_preds, mode='markers', marker=dict(symbol="x", size=10), name='Predictions'))
fig.update_layout(
    title="Graduation Rate Predictions for the Next 5 Years (4-Year Cohort)",
    xaxis_title="Cohort Year",
    yaxis_title="% Graduates",
    xaxis=dict(
        tickvals=np.append(six_year_graduates['Cohort Year'].values,future_years), 
        ticktext=np.append(six_year_graduates['Cohort Year'].values,future_years)
    )
)

#### Five Years

In [60]:
five_year_graduates

,Cohort Year,Duration,# Grads,Category,Cohort,# Total Cohort,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,...,# Still Enrolled,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month
0,2001,5,41317,All Students,5 year June,74143,55.726097,23925,32.268723,57.905945,...,8335,11.241790,21382,28.838865,2739.0,3.694212,294.0,0.396531,2006,June
1,2002,5,43835,All Students,5 year June,74511,58.830238,27478,36.877777,62.685070,...,8927,11.980782,18434,24.739971,1814.0,2.434540,1120.0,1.503134,2007,June
2,2003,5,46265,All Students,5 year June,73888,62.615040,29887,40.449059,64.599586,...,8520,11.530966,16095,21.782969,2129.0,2.881388,781.0,1.057005,2008,June
3,2004,5,49959,All Students,5 year June,75625,66.061485,33539,44.349091,67.133049,...,7365,9.738843,13786,18.229422,2104.0,2.782149,2401.0,3.174876,2009,June
4,2005,5,52988,All Students,5 year June,78147,67.805550,37611,48.128529,70.980225,...,6666,8.530078,14259,18.246382,2122.0,2.715395,2040.0,2.610465,2010,June
5,2006,5,55708,All Students,5 year August,79156,70.377480,42189,53.298550,75.732391,...,5650,7.137804,13877,17.531204,2228.0,2.814695,1611.0,2.035222,2011,August
6,2007,5,56153,All Students,5 year August,78922,71.149994,46104,58.417171,82.104248,...,4962,6.287220,14467,18.330757,1949.0,2.469527,1282.0,1.624389,2012,August
7,2008,5,56007,All Students,5 year August,79362,70.571556,52678,66.376854,94.056099,...,7700,9.702376,12572,15.841334,2008.0,2.530178,881.0,1.110103,2013,August
8,2009,5,56781,All Students,5 year August,78315,72.503349,53204,67.935898,93.700356,...,7540,9.627786,11731,14.979251,1228.0,1.568027,815.0,1.040669,2014,August
9,2010,5,56132,All Students,5 year August,75242,74.601952,52054,69.182106,92.734985,...,6602,8.774355,10495,13.948327,1216.0,1.616119,614.0,0.816034,2015,August


In [153]:
x = five_year_graduates['Cohort Year'].values
y = five_year_graduates['% Grads'].values
np.corrcoef(x,y)[0,1]

0.9864825772178978

In [154]:
x = x.reshape(-1,1)
model.fit(x,y)
future_years = get_future_years(five_year_graduates)

In [155]:
current_trend = model.predict(x)
five_year_grad_preds = model.predict(future_years.reshape(-1,1))
r2_score(y, current_trend)

0.9731478751544649

In [156]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=five_year_graduates['Cohort Year'], y=five_year_graduates['% Grads'], mode='markers', name='Current Data'))
fig.add_trace(go.Scatter(x=five_year_graduates['Cohort Year'], y=current_trend, mode='lines', name='Current Trend'))
fig.add_trace(go.Scatter(x=future_years, y=five_year_grad_preds, mode='markers', marker=dict(symbol="x", size=10), name='Predictions'))
fig.update_layout(
    title="Graduation Rate Predictions for the Next 5 Years (5-Year Cohort)",
    xaxis_title="Cohort Year",
    yaxis_title="% Graduates",
    xaxis=dict(
        tickvals=np.append(six_year_graduates['Cohort Year'].values,future_years), 
        ticktext=np.append(six_year_graduates['Cohort Year'].values,future_years)
    )
)

#### Six Years

In [67]:
six_year_graduates

,Cohort Year,Duration,# Grads,Category,Cohort,# Total Cohort,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,...,# Still Enrolled,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month
0,2001,6,43351,All Students,6 year June,74143,58.469444,24338,32.825756,56.141727,...,3896,5.254711,23538,31.746759,2988.0,4.030050,294.0,0.396531,2007,June
1,2002,6,46067,All Students,6 year June,74511,61.825771,27937,37.493793,60.644279,...,4729,6.346714,20140,27.029566,2042.0,2.740535,1152.0,1.546080,2008,June
2,2003,6,48586,All Students,6 year June,74027,65.632812,30483,41.178219,62.740295,...,4120,5.565537,17517,23.662989,2256.0,3.047537,1450.0,1.958745,2009,June
3,2004,6,52379,All Students,6 year June,75661,69.228531,34231,45.242596,65.352524,...,4468,5.905288,15306,20.229710,2156.0,2.849553,1253.0,1.656071,2010,June
4,2005,6,55277,All Students,6 year June,77994,70.873398,38312,49.121727,69.309113,...,3425,4.391364,15090,19.347643,2252.0,2.887402,1859.0,2.383517,2011,June
5,2006,6,57121,All Students,6 year June,78046,73.188889,42689,54.697231,74.734337,...,2484,3.182738,14618,18.729980,2174.0,2.785537,1541.0,1.974477,2012,June
6,2007,6,57651,All Students,6 year June,78998,72.977798,46967,59.453403,81.467796,...,2787,3.527937,15040,19.038456,2148.0,2.719056,1228.0,1.554470,2013,June
7,2008,6,57575,All Students,6 year June,79233,72.665428,53861,67.977989,93.549286,...,4351,5.491399,13545,17.095150,2699.0,3.406409,869.0,1.096765,2014,June
8,2009,6,58505,All Students,6 year June,78287,74.731438,54516,69.636086,93.181778,...,4541,5.800452,12818,16.373089,1332.0,1.701432,850.0,1.085749,2015,June
9,2010,6,57633,All Students,6 year June,75254,76.584633,53141,70.615517,92.205856,...,4022,5.344566,11446,15.209823,1309.0,1.739442,617.0,0.819890,2016,June


In [157]:
x = six_year_graduates['Cohort Year'].values
y = six_year_graduates['% Grads'].values
np.corrcoef(x,y)[0,1]

0.9813009073869123

In [158]:
x = x.reshape(-1,1)
model.fit(x,y)
future_years = get_future_years(six_year_graduates)

In [159]:
current_trend = model.predict(x)
six_year_grad_preds = model.predict(future_years.reshape(-1,1))
r2_score(y, current_trend)

0.9629514708383771

In [160]:
six_year_grad_preds

array([90.04403641, 91.57379937, 93.10356233, 94.63332529, 96.16308824])

In [161]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=six_year_graduates['Cohort Year'], y=six_year_graduates['% Grads'], mode='markers', name='Current Data'))
fig.add_trace(go.Scatter(x=six_year_graduates['Cohort Year'], y=current_trend, mode='lines', name='Current Trend'))
fig.add_trace(go.Scatter(x=future_years, y=six_year_grad_preds, mode='markers', marker=dict(symbol="x", size=10), name='Predictions'))
fig.update_layout(
    title="Graduation Rate Predictions for the Next 5 Years (6-Year Cohort)",
    xaxis_title="Cohort Year",
    yaxis_title="% Graduates",
    xaxis=dict(
        tickvals=np.append(six_year_graduates['Cohort Year'].values,future_years), 
        ticktext=np.append(six_year_graduates['Cohort Year'].values,future_years)
    )
)

In [72]:
five_year_grad_preds

array([89.25294442, 90.91629499, 92.57964556, 94.24299613, 95.9063467 ])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=future_years, y=four_year_grad_preds, mode='lines+markers',name="4 Years", line=dict(dash="dot")))
fig.add_trace(go.Scatter(x=future_years, y=five_year_grad_preds, mode='lines+markers',name="5 Years", line=dict(dash="dot")))
fig.add_trace(go.Scatter(x=future_years, y=six_year_grad_preds, mode='lines+markers',name="6 Years", line=dict(dash="dot")))
fig.update_layout(
    title="Graduation Rate Predictions for the Next 5 Years",
    xaxis_title="Cohort Year",
    yaxis_title="% Graduates"
)

### Students Who Did Not Graduate

In [283]:
all_graduates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 0 to 53
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Cohort Year                           54 non-null     int64  
 1   Duration                              54 non-null     int64  
 2   # Grads                               54 non-null     int64  
 3   Category                              54 non-null     object 
 4   Cohort                                54 non-null     object 
 5   # Total Cohort                        54 non-null     int64  
 6   % Grads                               54 non-null     float64
 7   # Total Regents                       54 non-null     int64  
 8   % Total Regents of Cohort             54 non-null     float64
 9   % Total Regents of Grads              54 non-null     float64
 10  # Advanced Regents                    54 non-null     int64  
 11  % Advanced Regents of

In [300]:
four_year_graduates

,Cohort Year,Duration,# Grads,Category,Cohort,# Total Cohort,% Grads,# Total Regents,% Total Regents of Cohort,% Total Regents of Grads,...,# Still Enrolled,% Still Enrolled,# Dropout,% Dropout,# SACC (IEP Diploma),% SACC (IEP Diploma) of Cohort,# TASC (GED),% TASC (GED) of Cohort,Graduation Year,Graduation Month
0,2001,4,34503,All Students,4 year June,74143,46.535748,22197,29.938093,64.333534,...,21094,28.450426,16337,22.034447,1895.0,2.555872,294.0,0.396531,2005,June
1,2002,4,36552,All Students,4 year June,74511,49.055843,25098,33.683617,68.663818,...,21721,29.151400,13758,18.464388,1010.0,1.355505,1102.0,1.478976,2006,June
2,2003,4,38990,All Students,4 year June,73888,52.769054,27445,37.144054,70.389847,...,21353,28.899145,11704,15.840191,1113.0,1.506334,633.0,0.856702,2007,June
3,2004,4,45490,All Students,4 year August,75009,60.700001,31976,42.599998,70.300003,...,17669,23.600000,10126,13.500000,NaN,NaN,NaN,NaN,2008,August
4,2005,4,48499,All Students,4 year August,77378,62.678020,35718,46.160408,73.646881,...,16464,21.277365,9155,11.831529,1336.0,1.726589,1924.0,2.486495,2009,August
5,2006,4,51006,All Students,4 year August,78346,65.103516,40151,51.248310,78.718193,...,14975,19.113930,9440,12.049115,1222.0,1.559748,1664.0,2.123912,2010,August
6,2007,4,52069,All Students,4 year August,79476,65.515373,44216,55.634407,84.918091,...,15403,19.380693,9631,12.118124,1000.0,1.258242,1272.0,1.600483,2011,August
7,2008,4,51573,All Students,4 year August,79719,64.693489,49004,61.470917,95.018715,...,17063,21.403931,9096,11.410078,1093.0,1.371066,781.0,0.979691,2012,August
8,2009,4,51970,All Students,4 year August,78721,66.017960,49258,62.572884,94.781609,...,16577,21.057913,8363,10.623594,948.0,1.204253,693.0,0.880324,2013,August
9,2010,4,51681,All Students,4 year August,75524,68.429901,48563,64.301414,93.966835,...,15127,20.029394,7309,9.677718,781.0,1.034108,484.0,0.640856,2014,August


In [165]:
four_year_not_graduates = four_year_graduates[['Cohort Year', '# Total Cohort', '% Still Enrolled', '% Dropout', '# Grads', '# Still Enrolled', '# Dropout']]
five_year_not_graduates = five_year_graduates[['Cohort Year', '# Total Cohort', '% Still Enrolled', '% Dropout', '# Grads', '# Still Enrolled', '# Dropout']]
six_year_not_graduates = six_year_graduates[['Cohort Year', '# Total Cohort', '% Still Enrolled', '% Dropout', '# Grads', '# Still Enrolled', '# Dropout']]

In [166]:
def update_df(df):
    df = df.rename(columns={'% Dropout': '% Dropout vs Graduates', '% Still Enrolled' : '% Still Enrolled vs Graduates'})
    df['Not Graduates'] = df['# Total Cohort'] - df['# Grads']
    df['% Dropout'] = df['# Dropout'] / df['Not Graduates'] * 100
    df['% Still Enrolled'] = df['# Still Enrolled'] / df['Not Graduates'] * 100

    return df

In [167]:
four_year_not_graduates = update_df(four_year_not_graduates)
five_year_not_graduates = update_df(five_year_not_graduates)
six_year_not_graduates = update_df(six_year_not_graduates)

In [163]:
def graph_not_grads(y_axis):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=four_year_not_graduates['Cohort Year'], y=four_year_not_graduates[y_axis], mode='lines+markers', name='After 4 years'))
    fig.add_trace(go.Scatter(x=five_year_not_graduates['Cohort Year'], y=five_year_not_graduates[y_axis], mode='lines+markers', name='After 5 years'))
    fig.add_trace(go.Scatter(x=six_year_not_graduates['Cohort Year'], y=six_year_not_graduates[y_axis], mode='lines+markers', name='After 6 years'))

    fig.update_layout(
        xaxis_title="Cohort Year",
        xaxis=dict(
            tickvals=four_year_not_graduates['Cohort Year'], 
            ticktext=four_year_not_graduates['Cohort Year'],
        )
    )

    return fig

In [168]:
fig = graph_not_grads('% Dropout vs Graduates')
fig.update_layout(
    title="% Dropouts Compared to Graduates",
    yaxis_title="% Dropped"
)

In [169]:
fig = graph_not_grads("% Dropout")
fig.update_layout(
    title="% Dropouts After Not Graduating",
    yaxis_title="% Dropped"
)

In [171]:
fig = graph_not_grads("% Still Enrolled vs Graduates")
fig.update_layout(
    title="% of Students who Still Enrollled Compared to Graduates",
    yaxis_title="% Enrolled"
)

In [172]:
fig = graph_not_grads("% Still Enrolled")
fig.update_layout(
    title="% of Students who Still Enrollled Despite Not Graduating",
    yaxis_title="% Enrolled"
)

## Types of Diploma

In [173]:
cohort_total_students_per_year

,Cohort Year,# Grads,# Total Cohort,Average Grad %,4-year Grads,5-year Grads,6-year Grads,4-year Grad %,5-year Grad %,6-year Grad %
0,2001,119171,222429,53.577097,34503,41317,43351,15.511916,18.575366,19.489815
1,2002,126454,223533,56.570618,36552,43835,46067,16.351948,19.610080,20.608590
2,2003,133841,221803,60.342286,38990,46265,48586,17.578662,20.858600,21.905024
3,2004,147828,226295,65.325350,45490,49959,52379,20.102079,22.076935,23.146336
4,2005,156764,233519,67.131154,48499,52988,55277,20.768760,22.691087,23.671307
5,2006,163835,235548,69.554825,51006,55708,57121,21.654185,23.650381,24.250259
6,2007,165873,237396,69.871860,52069,56153,57651,21.933394,23.653726,24.284739
7,2008,165155,238314,69.301426,51573,56007,57575,21.640776,23.501347,24.159302
8,2009,167256,235323,71.075076,51970,56781,58505,22.084539,24.128963,24.861573
9,2010,165446,226020,73.199717,51681,56132,57633,22.865676,24.834970,25.499071
